# Chatbot project based on techwithtims tutorials

# Setup

In [1]:
import numpy as np
import random
import json
import nltk
from nltk.stem.lancaster import LancasterStemmer
import tensorflow as tf
from tensorflow import keras
import pickle

# Read and parse intents.json

In [2]:
with open('intents.json') as file:
    data = json.load(file)

words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

stemmer = LancasterStemmer()
words = [stemmer.stem(w.lower()) for w in words if w != '?']
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    training.append(bag)
    output.append(output_row)

training = np.array(training)
output = np.array(output)

# Define model and train

In [3]:
model = keras.models.Sequential([
    
    keras.layers.Dense(len(words),input_shape=[len(words)]),
    keras.layers.Dense(16, activation='sigmoid'),
    keras.layers.Dense(16, activation='sigmoid'),
    keras.layers.Dense(len(labels), activation='softmax')
    
])

In [5]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')
history = model.fit(x=training,
                    y=output,
                    batch_size=8,
                    epochs=400,
                    callbacks=[keras.callbacks.ReduceLROnPlateau(monitor='loss',factor=0.7,patience=30),
                               keras.callbacks.EarlyStopping(monitor='loss', patience=150, restore_best_weights=True)])

Epoch 1/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0991 - accuracy: 1.0000
Epoch 2/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0969 - accuracy: 1.0000
Epoch 3/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0948 - accuracy: 1.0000
Epoch 4/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0929 - accuracy: 1.0000
Epoch 5/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0911 - accuracy: 1.0000
Epoch 6/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0892 - accuracy: 1.0000
Epoch 7/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0875 - accuracy: 1.0000
Epoch 8/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0857 - accuracy: 1.0000
Epoch 9/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0841 - accuracy: 1.0000
Epoch 10/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0824 - accuracy: 1.0000
Epoch 11/400
5/5 [===========

5/5 [==============================] - 0s 2ms/step - loss: 0.0092 - accuracy: 1.0000
Epoch 168/400
5/5 [==============================] - 0s 2ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 169/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0090 - accuracy: 1.0000
Epoch 170/400
5/5 [==============================] - 0s 2ms/step - loss: 0.0089 - accuracy: 1.0000
Epoch 171/400
5/5 [==============================] - 0s 2ms/step - loss: 0.0088 - accuracy: 1.0000
Epoch 172/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0087 - accuracy: 1.0000
Epoch 173/400
5/5 [==============================] - 0s 2ms/step - loss: 0.0087 - accuracy: 1.0000
Epoch 174/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0086 - accuracy: 1.0000
Epoch 175/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0085 - accuracy: 1.0000
Epoch 176/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0084 - accuracy: 1.0000
Epoch 177/400
5/5 [=====

5/5 [==============================] - 0s 3ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 332/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 333/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 334/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 335/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 336/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 337/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 338/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 339/400
5/5 [==============================] - 0s 3ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 340/400
5/5 [==============================] - 0s 2ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 341/400
5/5 [=====

# Setup predictions

In [6]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(w.lower()) for w in s_words]
    
    for se in s_words:
        for i,w in enumerate(words):
            if w == se:
                bag[i] = 1
    
    return np.array(bag).reshape(1,-1)

def model_response(pred, labels, data):
    if max(pred)<0.9:
        uncertain_responses = ['I dont understand', 'Sorry I dont get what you\'re saying', 'Could you word that differently?']
        print(f"Chatbot: {random.choice(uncertain_responses)}")
    else:
        pred_idx = np.argmax(pred)
        label = labels[pred_idx]

        for intent in data['intents']:
            if label == intent['tag']:
                print(f"Chatbot: {random.choice(intent['responses'])}")
                break

#  Define chat loop and chat!

In [7]:
def chat():
    print('Start talking with the bot! (type quit to stop)\n')
    while True:
        inp = input("You: ")
        if inp.lower()=='quit':
            break
        bag = bag_of_words(inp, words)
        pred = model.predict(bag).reshape(-1)
        model_response(pred,labels,data)

In [8]:
chat()

Start talking with the bot! (type quit to stop)

You: hello
Chatbot: Good to see you again!
You: what do you sell?
Chatbot: Cookies are on the menu!
You: what time do you open?
Chatbot: Could you word that differently?
You: quit
